In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
base_dir = "../input/meat-freshness-image-dataset/Meat Freshness.v1-new-dataset.multiclass"
import tensorflow as tf

**CREATE SUBDIRECTORIES WITH CLASS NAMES**

In [ ]:
"""Define paths."""
base_kaggle_path = "../outputs"
paths = ["train","val"]
"""Create new subdirectories with class names for train and val."""
classes = ["FRESH","HALF-FRESH","SPOILED"]
for i in range(len(paths)):
    for j in range(len(classes)):
        sub_folder = os.path.join(paths[i],classes[j])
        os.makedirs(os.path.join(base_kaggle_path,sub_folder))

In [ ]:
os.listdir(base_kaggle_path+"/train")

**DISTRIBUTE IMAGES TO FOLDERS ACCORDING TO THEIR CLASSES.**

In [ ]:
import shutil
train_path = os.path.join(base_dir,"train")
"""For train."""
for filename in os.listdir(train_path):
    if filename.rfind("FRESH") == 0:
        shutil.copy(os.path.join(train_path,filename),base_kaggle_path + "/train/FRESH")
    elif filename.rfind("HALF-FRESH") == 0:
        shutil.copy(os.path.join(train_path,filename),base_kaggle_path + "/train/HALF-FRESH")
    elif filename.rfind("SPOILED") == 0:
        shutil.copy(os.path.join(train_path,filename),base_kaggle_path + "/train/SPOILED")
    else:
        pass


In [ ]:
val_path = os.path.join(base_dir,"valid")
"""For validation."""
for filename in os.listdir(val_path):
    if filename.rfind("FRESH") == 0:
        shutil.copy(os.path.join(val_path,filename),base_kaggle_path + "/val/FRESH")
    elif filename.rfind("HALF-FRESH") == 0:
        shutil.copy(os.path.join(val_path,filename),base_kaggle_path + "/val/HALF-FRESH")
    elif filename.rfind("SPOILED") == 0:
        shutil.copy(os.path.join(val_path,filename),base_kaggle_path + "/val/SPOILED")
    else:
        pass

In [ ]:
print("Training : \n")
print(len(os.listdir(base_kaggle_path+"/train/FRESH")))
print(len(os.listdir(base_kaggle_path+"/train/HALF-FRESH")))
print(len(os.listdir(base_kaggle_path+"/train/SPOILED")))
print("Validation : \n")
print(len(os.listdir(base_kaggle_path+"/val/FRESH")))
print(len(os.listdir(base_kaggle_path+"/val/HALF-FRESH")))
print(len(os.listdir(base_kaggle_path+"/val/SPOILED")))

**PRE PROCESSING OF IMAGES**

In [ ]:
"""All images will be scaled to 1./255 to obtain 0-1 normalized image.Also image augmentation is used."""
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,rotation_range = 40,
                                                                 width_shift_range = 0.2,height_shift_range = 0.2,
                                                                 shear_range = 0.2,zoom_range = 0.2,
                                                                 horizontal_flip = True,vertical_flip = True,
                                                                 fill_mode = "nearest",)
"""Validation images also will be scale dto 1./255 to obtain 0-1 normalized image,but image augmentation is NOT USED."""
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(os.path.join(base_kaggle_path,"train"),target_size = (150,150),
                                                   class_mode = "categorical",batch_size = 32,seed = 42)
validation_generator = validation_datagen.flow_from_directory(os.path.join(base_kaggle_path,"val"),target_size = (150,150),
                                                    class_mode = "categorical",batch_size = 32,seed = 42,shuffle = False)

**TRANSFER LEARNING MODEL**

**(1)Use Xception as base model.**

In [ ]:
from tensorflow.keras.applications.xception import Xception
base_model = Xception(input_shape = (150,150,3),weights = "imagenet",include_top = False,pooling = "max")
"""Freeze layers to stop updating weights of base model."""
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
base_model.summary()

**(2)Create an architecture to feed models.**

In [ ]:
"""Here,we can assign last layer as add_11.It means that we can start to update weights after this layer"""
last_layer = base_model.get_layer("add_11")
print(last_layer.output_shape)

In [ ]:
"""GlobalAveragePooling layer to reduce input dim to 1D."""
x = tf.keras.layers.BatchNormalization()(last_layer.output)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
"""Fully connected layer."""
x = tf.keras.layers.Dense(128,activation = "relu")(x)
"""Add dropout layer."""
x = tf.keras.layers.Dropout(0.3)(x)
"""Output layer"""
x = tf.keras.layers.Dense(3,activation = "softmax")(x)
"""Here,we can connect model end to end."""
model = tf.keras.models.Model(base_model.input,x)

In [ ]:
model.summary()

**CNN ARCHITECTURE**

In [ ]:
model.summary()

**COMPILE AND FIT**

In [ ]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate =0.0001),loss = "categorical_crossentropy",metrics = ["acc"])
"""Callback"""
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs = {}):
        if epoch >= 10 and logs.get("acc") - logs.get("val_acc") >= 0.1:
            print("Model tends to be overfitting.Stop it.")
            self.model.stop_training = True
        elif logs.get("acc") > 0.9:
            print("Model tends to be overfitting.Stop it.")
            self.model.stop_training = True
callback = myCallback()

In [ ]:
history = model.fit(train_generator,epochs = 50,batch_size = 32,validation_data = validation_generator,
                   callbacks = [callback,],verbose = 1)

**PLOT RESULTS**

In [ ]:
import matplotlib.pyplot as plt
"""Accuracies."""
acc = history.history["acc"]
val_acc = history.history["val_acc"]
epochs = range(29)
plt.plot(epochs,acc,label = "Training accuracy")
plt.plot(epochs,val_acc,label = "Validation accuracy")
plt.legend()
plt.show()

In [ ]:
"""Losses."""
loss = history.history["loss"]
val_loss = history.history["val_loss"]
plt.plot(epochs,loss,label = "Training Loss")
plt.plot(epochs,val_loss,label = "Validation Loss")
plt.legend()
plt.show()

**SAVE MODEL AND LOAD MODEL**

In [ ]:
model.save("meat_classify.h5")

In [ ]:
model1 = tf.keras.models.load_model("meat_classify.h5")

**GET THE IMAGES FROM INTERNET AND PROCESS THEM**

In [ ]:
from PIL import Image
import requests
from io import BytesIO
import numpy as np 

In [ ]:
def get_and_process(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img1 = img
    """Resize img to proper for feed model."""
    img = img.resize((150,150))
    """Convert img to numpy array,rescale it,expand dims and check vertically."""
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = x / 255.0 
    x = np.expand_dims(x,axis = 0)
    img_tensor = np.vstack([x])
    return img1,img_tensor

**FINAL : PREDICT IMAGE**

In [1]:
import matplotlib.pyplot as plt 
url = "https://media.istockphoto.com/photos/spoiled-steak-picture-id466978127"
img1,test_img = get_and_process(url)
"""Predict."""
pred = model1.predict(test_img)
classes = list(train_generator.class_indices.keys())
print(f"Prediction is : {classes[np.argmax(pred)]}")
plt.imshow(img1)
plt.show()

print(classes)
print(pred)

NameError: name 'get_and_process' is not defined